In [3]:
import pandas as pd
import numpy as np
import scipy.stats as sc

In [4]:
data = pd.read_csv('diabetes.txt', sep = " ")

In [5]:
# This is the good data
data.shape

(442, 11)

In [6]:
# Define X and Y arrays
X = data.iloc[:,:data.shape[1]-1].as_matrix()
Y = data.iloc[:,data.shape[1]-1].ravel()

In [7]:
# We center the Y
Y_tilde = Y - Y.mean(axis=0)

## 1] Empirical Bayes by Marginal Maximum Likelihood

The Gibbs sampler used here uses the following full conditional distributions:

- The full conditional for $\beta$ is:

<h3 align="center"> $\mathcal{N}_p(A^{-1}X^T\tilde{y}, \sigma^2A^{-1})$ where $A = X^TX+D^{-1}_\tau$ and $D_\tau = diag(\tau^2_1,...,\tau^2_p)$ </h3>

- The full conditional for $\sigma^2$ is:

<h3 align="center"> $\mathcal{I}nverse\mathcal{G}amma(\frac{n-1+p}{2}, \frac{(\tilde{y}-X\beta)^T(\tilde{y}-X\beta) + \beta^TD^{-1}_\tau\beta}{2})$ </h3>

- $\tau^2_1, ..., \tau^2_p$ are conditionnaly independent and $\frac{1}{\tau^2_j}$ has as conditional distribution:

<h3 align="center"> $\mathcal{I}nverse\mathcal{G}ausian(\sqrt{\frac{\lambda^2\sigma^2}{\beta^2_j}}, \lambda^2)$ </h3>

In [38]:
# Initialization
beta = np.random.uniform(size = X.shape[1])
sigma_sq = np.random.uniform()
tau_sq = np.random.uniform(size = X.shape[1])

beta_ = []
sigma_sq_ = []
tau_sq_ = []

#### Full conditional for $\beta$

In [9]:
D_tau = np.diag(tau_sq)
A = X.transpose().dot(X) + np.linalg.inv(D_tau)
multi_norm_mean = np.linalg.inv(A).dot(X.transpose()).dot(Y_tilde)
multi_norm_cov = sigma_sq * np.linalg.inv(A)
beta_.append(np.random.multivariate_normal(multi_norm_mean, multi_norm_cov))

#### Full conditional for $\sigma^2$

In [10]:
shape = (X.shape[0]-1+X.shape[1])/2
scale = ((Y_tilde - X.dot(beta)).dot((Y_tilde - X.dot(beta))) + beta.transpose().dot(np.linalg.inv(D_tau)).dot(beta))/2
sigma_sq_.append(sc.invgamma.rvs(a = shape, scale = scale))

#### Full conditional for $\tau^2_1, ..., \tau^2_p$

In [11]:
###############
# Say lambda 0 is equal to 1
lambda_ = 2

In [32]:
mean = np.sqrt(lambda_**2*sigma_sq/beta**2)
scale = lambda_**2
tau_sq_.append(np.random.wald(mean, scale))

In [42]:
t = []
for i in range(len(tau_sq)):
    t.append(np.random.wald(mean[i], scale))
tau_sq_.append(np.array(t))

#### Gibbs sampler

## 1] CHOOSING THE BAYESIAN LASSO PARAMETER

The Gibbs sampler used here uses the following full conditional distributions:

- The full conditional for $\beta$ is:

<h3 align="center"> $\mathcal{N}_p(A^{-1}X^T\tilde{y}, \sigma^2A^{-1})$ where $A = X^TX+D^{-1}_\tau$ and $D_\tau = diag(\tau^2_1,...,\tau^2_p)$ </h3>

- The full conditional for $\sigma^2$ is:

<h3 align="center"> $\mathcal{I}nverse\mathcal{G}amma(\frac{n-1+p}{2}, \frac{(\tilde{y}-X\beta)^T(\tilde{y}-X\beta) + \beta^TD^{-1}_\tau\beta}{2})$ </h3>

- $\tau^2_1, ..., \tau^2_p$ are conditionnaly independent and $\frac{1}{\tau^2_j}$ has as conditional distribution:

<h3 align="center"> $\mathcal{I}nverse\mathcal{G}ausian(\sqrt{\frac{\lambda^2\sigma^2}{\beta^2_j}}, \lambda^2)$ </h3>

In [64]:
def Gibbs_sampler(n, lambda_):
    # Initialization
    beta = [np.random.uniform(size = X.shape[1])]
    sigma_sq = [np.random.uniform()]
    tau_sq = [np.random.uniform(size = X.shape[1])]
    for i in range(n):
        # Full conditional for beta
        D_tau = np.diag(tau_sq[i])
        A = X.transpose().dot(X) + np.linalg.inv(D_tau)
        multi_norm_mean = np.linalg.inv(A).dot(X.transpose()).dot(Y_tilde)
        multi_norm_cov = sigma_sq[i] * np.linalg.inv(A)
        beta.append(np.random.multivariate_normal(multi_norm_mean, multi_norm_cov))
        # Full conditional for sigma_sq
        shape = (X.shape[0]-1+X.shape[1])/2
        scale = ((Y_tilde - X.dot(beta[i+1])).dot((Y_tilde - X.dot(beta[i+1]))) + beta[i+1].transpose().dot(np.linalg.inv(D_tau)).dot(beta[i+1]))/2
        sigma_sq.append(sc.invgamma.rvs(a = shape, scale = scale))
        # Full conditional for tau_1,...,tau_p
        mean = np.sqrt(lambda_**2*sigma_sq[i+1]/beta[i+1]**2)
        scale = np.repeat(lambda_**2, X.shape[1])
        tau_sq.append(1/np.random.wald(mean, scale))
    return tau_sq[int(n/2):]

#### Empirical Bayes by Marginal Maximum Likelihood

For the Bayesian Lasso, each iteration of the algorithm involves running the Gibbs sampler using a $\lambda$ value estimated from the sample of the previous iteration. Specifically, iteration $k$ uses the Gibbs sampler of Section 2 with hyperparameter $\lambda^{(k-1)}$ (i.e., the estimate from iteration $k-1$) to approximate the ideal updated estimate:

<h3 align="center"> $\lambda^{(k)} = \sqrt{\frac{2p}{\sum\limits^p_{j=1}E_{\lambda^{(k-1)}}[\tau^2_j|\tilde{y}]}}$ </h3>

by replacing the conditional expectations with averages from the Gibbs sample. We suggest the initial value:

<h3 align="center"> $\lambda^{(0)} = \frac{p\sqrt{\hat{\sigma}^2_{LS}}}{\sum\limits^p_{j=1}|\hat{\beta}^{LS}_j|}$ </h3>

where $\hat{\sigma}^2_{LS}$ and $\hat{\beta}^{LS}_j$ are estimates from the usual least squares procedure.

In [65]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X,Y_tilde)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [66]:
# Lambda_init
lambda_init = (X.shape[1]*np.sqrt((np.sum((Y_tilde - lm.predict(X))**2))/(X.shape[0]-X.shape[1])))/np.sum(np.abs(lm.coef_))

In [68]:
# This will be used to find the next k lambda
for i in range(50):
    if i==0:
        lambda_ = np.sqrt(2*X.shape[1]/sum(np.mean(Gibbs_sampler(1000, lambda_init), axis=0)))
    else:
        lambda_ = np.sqrt(2*X.shape[1]/sum(np.mean(Gibbs_sampler(1000, lambda_), axis=0)))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [69]:
lambda_

0.23821022738257597

#### Hyperpriors for the Lasso Parameter

We need to modify the previous Gibbs sampler by introducing the full conditional distribution of $\lambda^2$, which is:

<h3 align="center"> $\mathcal{G}amma(p+r, \frac{\sum\limits_{j=1}^p\tau^2_j}{2}+\delta)$ </h3>

In [26]:
def Gibbs_sampler_bis(n, r, delta):
    # Initialization
    beta = [np.random.uniform(size = X.shape[1])]
    sigma_sq = [np.random.uniform()]
    tau_sq = [np.random.uniform(size = X.shape[1])]
    lambda_sq = [np.random.uniform()]
    for i in range(n):
        # Full conditional for beta
        D_tau = np.diag(tau_sq[i])
        A = X.transpose().dot(X) + np.linalg.inv(D_tau)
        multi_norm_mean = np.linalg.inv(A).dot(X.transpose()).dot(Y_tilde)
        multi_norm_cov = sigma_sq[i] * np.linalg.inv(A)
        beta.append(np.random.multivariate_normal(multi_norm_mean, multi_norm_cov))
        # Full conditional for sigma_sq
        shape = (X.shape[0]-1+X.shape[1])/2
        scale = ((Y_tilde - X.dot(beta[i+1])).dot((Y_tilde - X.dot(beta[i+1]))) + beta[i+1].transpose().dot(np.linalg.inv(D_tau)).dot(beta[i+1]))/2
        sigma_sq.append(sc.invgamma.rvs(a = shape, scale = scale))
        # Full conditional for tau_1,...,tau_p
        mean = np.sqrt(lambda_sq[i]*sigma_sq[i+1]/beta[i+1]**2)
        scale = np.repeat(lambda_sq[i], X.shape[1])
        tau_sq.append(1/np.random.wald(mean, scale))
        # Full conditional for lambda_sq
        shape = X.shape[1] + r
        rate = sum(tau_sq[i+1])/2+delta
        lambda_sq.append(np.random.gamma(shape, 1/rate))
    return lambda_sq

In [24]:
lambda_sq_bis = Gibbs_sampler_bis(10000, 1, 1.78)

In [25]:
np.median([np.sqrt(i) for i in lambda_sq_bis])

0.27650733947097222

# 2] Test on largest dataset

In [27]:
train = pd.read_csv('digits_train.csv')
train = train.sample(3000)

In [28]:
X = train.iloc[:,1:].as_matrix()
Y_tilde = train.iloc[:,0].ravel()

#### Empirical Bayes by Marginal Maximum Likelihood

In [42]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X,Y_tilde)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [43]:
# Lambda_init
lambda_init = (X.shape[1]*np.sqrt((np.sum((Y_tilde - lm.predict(X))**2))/(X.shape[0]-X.shape[1])))/np.sum(np.abs(lm.coef_))

In [46]:
# This will be used to find the next k lambda
for i in range(20):
    if i==0:
        lambda_ = np.sqrt(2*X.shape[1]/sum(np.mean(Gibbs_sampler_corrected(5, lambda_init), axis=0)))
    else:
        lambda_ = np.sqrt(2*X.shape[1]/sum(np.mean(Gibbs_sampler_corrected(5, lambda_), axis=0)))
    print(i)

/Users/Linger/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: RuntimeWarning: covariance is not positive-semidefinite.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [47]:
lambda_

0.0010760539836392866

#### Problem encountered in large dimansion with sparse dataset: Theory vs Implementation

We have the problem that $\sigma^2A^{-1}$ which is the covariance matrix used to generate the $\beta$'s is from the computer point of view not positive semi-definite.

Theoretically, the matrix is positive semidefinite, with several eigenvalues being exactly zero. But the computations with floating point numbers introduce truncation errors which result in some of those eigenvalues being very small but negative; hence, the matrix is not positive semidefinite.

A way to correct for the floating point errors is to add a tiny multiple of the identity matrix to the covariance matrix. I take this into account and add it in my Gibbs sampler.

In [45]:
def Gibbs_sampler_corrected(n, lambda_):
    # Initialization
    beta = [np.random.uniform(size = X.shape[1])]
    sigma_sq = [np.random.uniform()]
    tau_sq = [np.random.uniform(size = X.shape[1])]
    for i in range(n):
        # Full conditional for beta
        D_tau = np.diag(tau_sq[i])
        A = X.transpose().dot(X) + np.linalg.inv(D_tau)
        multi_norm_mean = np.linalg.inv(A).dot(X.transpose()).dot(Y_tilde)
        min_eig = np.min(np.real(np.linalg.inv(A)))
        if min_eig < 0:
            A -= 10*min_eig * np.eye(*A.shape)
        multi_norm_cov = sigma_sq[i] * np.linalg.inv(A)
        beta.append(np.random.multivariate_normal(multi_norm_mean, multi_norm_cov))
        # Full conditional for sigma_sq
        shape = (X.shape[0]-1+X.shape[1])/2
        scale = ((Y_tilde - X.dot(beta[i+1])).dot((Y_tilde - X.dot(beta[i+1]))) + beta[i+1].transpose().dot(np.linalg.inv(D_tau)).dot(beta[i+1]))/2
        sigma_sq.append(sc.invgamma.rvs(a = shape, scale = scale))
        # Full conditional for tau_1,...,tau_p
        mean = np.sqrt(lambda_**2*sigma_sq[i+1]/beta[i+1]**2)
        scale = np.repeat(lambda_**2, X.shape[1])
        tau_sq.append(1/np.random.wald(mean, scale))
    return tau_sq[int(n/2):]

#### Hyperpriors for the Lasso Parameter

Same problem as before, so I need to modify the Gibbs sampler

In [48]:
def Gibbs_sampler_bis_corrected(n, r, delta):
    # Initialization
    beta = [np.random.uniform(size = X.shape[1])]
    sigma_sq = [np.random.uniform()]
    tau_sq = [np.random.uniform(size = X.shape[1])]
    lambda_sq = [np.random.uniform()]
    for i in range(n):
        # Full conditional for beta
        D_tau = np.diag(tau_sq[i])
        A = X.transpose().dot(X) + np.linalg.inv(D_tau)
        multi_norm_mean = np.linalg.inv(A).dot(X.transpose()).dot(Y_tilde)
        min_eig = np.min(np.real(np.linalg.inv(A)))
        if min_eig < 0:
            A -= 10*min_eig * np.eye(*A.shape)
        multi_norm_cov = sigma_sq[i] * np.linalg.inv(A)
        beta.append(np.random.multivariate_normal(multi_norm_mean, multi_norm_cov))
        # Full conditional for sigma_sq
        shape = (X.shape[0]-1+X.shape[1])/2
        scale = ((Y_tilde - X.dot(beta[i+1])).dot((Y_tilde - X.dot(beta[i+1]))) + beta[i+1].transpose().dot(np.linalg.inv(D_tau)).dot(beta[i+1]))/2
        sigma_sq.append(sc.invgamma.rvs(a = shape, scale = scale))
        # Full conditional for tau_1,...,tau_p
        mean = np.sqrt(lambda_sq[i]*sigma_sq[i+1]/beta[i+1]**2)
        scale = np.repeat(lambda_sq[i], X.shape[1])
        tau_sq.append(1/np.random.wald(mean, scale))
        # Full conditional for lambda_sq
        shape = X.shape[1] + r
        rate = sum(tau_sq[i+1])/2+delta
        lambda_sq.append(np.random.gamma(shape, 1/rate))
    return lambda_sq

In [49]:
lambda_sq_bis = Gibbs_sampler_bis_corrected(100, 1, 1.78)

KeyboardInterrupt: 

In [36]:
np.median([np.sqrt(i) for i in lambda_sq_bis])

2.6366140491475503